In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, InvalidSelectorException
import numpy as np
import pandas as pd
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [26]:
def pos_input(pos):
    driver.find_element(By.XPATH, '//*[@name="jobTitleAutocomplete"]').send_keys(pos)
    driver.find_element(By.XPATH, '//*[@name="jobTitleAutocomplete"]').click()
    sleep(1)
    driver.find_element(By.XPATH, '//*[@name="jobTitleAutocomplete"]').send_keys(Keys.DOWN)
    driver.find_element(By.XPATH, '//*[@name="jobTitleAutocomplete"]').send_keys(Keys.ENTER)
    
def comp_input(comp):
    driver.find_element(By.XPATH, '//*[@name="employerAutocomplete"]').send_keys(comp)
    driver.find_element(By.XPATH, '//*[@name="employerAutocomplete"]').click()
    sleep(1)
    driver.find_element(By.XPATH, '//*[@name="employerAutocomplete"]').send_keys(Keys.DOWN)
    driver.find_element(By.XPATH, '//*[@name="employerAutocomplete"]').send_keys(Keys.ENTER)
    sleep(1)

In [63]:
df=pd.read_csv(r"C:\Users\arman\OneDrive\Desktop\Linkedin_Scraper\items.csv")

In [64]:
df=df.replace({"Company Name\n": ''}, regex=True)
df=df.replace({"About\n": ''}, regex=True)
df = df.drop(columns='small_bio', axis=1)
positions = df['job_title']

In [65]:
new_pos = positions.str.split(' at ', 1, expand = True).rename(columns = {0 : 'pos', 1 : 'company'})

In [66]:
new_pos['pos'] =  new_pos['pos'].str.split(',', 1, expand = True)[0].str.split('-', 1, expand = True)[0].str.split(' in ', 1, expand = True)[0]
new_pos['pos'] =  new_pos['pos'].str.split(' II', 1, expand = True)[0].str.split(' for ', 1, expand = True)[0].str.split(' and ', 1, expand = True)[0]
new_pos['pos'] =  new_pos['pos'].apply(lambda x: x.split("|")[0].strip() if "|" in str(x) else x)

In [67]:
driver = webdriver.Chrome(service=Service("C:\\chromedriver.exe"))
driver.maximize_window()
salaries=[]
for request in new_pos.to_dict('records')[:5]:
    try:
        driver.get('https://www.glassdoor.com/Salaries/know-your-worth.htm')
        pos_input(request['pos'])
        comp_input(request['company'])
        driver.find_element(By.XPATH, '//*[@class="gd-ui-button mt-xl css-1kp7eb7"]').click()
        sleep(4)
        try:
            salary = driver.find_element(By.XPATH, '//*[@class="d-inline mb-0 mt-sm css-1tx26uv"]').text
        except NoSuchElementException:
            try:
                salary = driver.find_element(By.XPATH, '//*[@data-test="occ-median-average-pay"]').text
            except NoSuchElementException:
                salary = np.NaN
                
        request.update({'salary' : salary })
        salaries.append(request)
 
        driver.delete_all_cookies()
    except TypeError:
        request.update({'salary' : np.NaN })
        salaries.append(request)

In [68]:
salaries

[{'pos': 'Software Engineer', 'company': 'Teradyne', 'salary': '$93,916'},
 {'pos': 'Associate Human Resources Director',
  'company': 'Harvard Kennedy School',
  'salary': '$135,955'},
 {'pos': 'Experienced regulatory affairs professional specializing',
  'company': None,
  'salary': nan},
 {'pos': 'Senior Integrated Producer',
  'company': 'Wayfair',
  'salary': '$91,586'},
 {'pos': 'Digital Learning Specialist', 'company': None, 'salary': nan}]

In [9]:
df['clean_pos']=new_pos
df['clean_comp']=new_comp

In [10]:
df1=pd.read_csv(r"C:\Users\arman\OneDrive\Desktop\linkedin_with_salaries.csv")

In [11]:
df.drop(df.tail(7).index,
         inplace = True)

In [13]:
df['salaries'] = df1['salary']

In [15]:
df = df.iloc[: , 1:]

In [17]:
df.drop("name", axis=1, inplace=True)
df

,job_title,loc,cur_comp,educ,about,skills,experiences.company_1.exp_positions,experiences.company_1.exp_companies,experiences.company_1.exp_years,experiences.company_1.exp_durations,...,licenses.license_3.licenses,licenses.license_3.issue_date,licenses.license_3.issue_authority,experiences.company_10.exp_positions,experiences.company_10.exp_companies,experiences.company_10.exp_years,experiences.company_10.exp_durations,clean_pos,clean_comp,salaries
0,Software Engineer at Teradyne,"Boston, Massachusetts, United States",Teradyne,Northeastern University,NaN,NaN,Software Engineer,Teradyne,Feb 2013 – Present,9 yrs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Engineer,Teradyne,"$93,916"
1,Associate Human Resources Director at Harvard ...,"Cambridge, Massachusetts, United States",Harvard Kennedy School,Northeastern University,"Currently, I work at the Harvard Kennedy Schoo...",Employee Relations,Harvard Kennedy School,NaN,Feb 2021 – Present,1 yr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Associate Human Resources Director,Harvard Kennedy School,"$135,955"
2,Experienced regulatory affairs professional sp...,"Cambridge, Massachusetts, United States",IQVIA,Northeastern University,NaN,NaN,IQVIA,NaN,Apr 2021 – Present,10 mos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Experienced regulatory affairs professional sp...,None,NaN
3,Senior Integrated Producer at Wayfair,"Boston, Massachusetts, United States",Wayfair,Northeastern University,NaN,NaN,Wayfair,NaN,2015 – Present,7 yrs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Senior Integrated Producer,Wayfair,"$91,586"
4,Digital Learning Specialist,"Somerville, Massachusetts, United States",Manchester-Essex Regional School District,Northeastern University,NaN,NaN,Digital Learning Specialist,Manchester-Essex Regional School District Full...,Nov 2021 – Present,3 mos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Digital Learning Specialist,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,Operation Lead at Broad Institute of MIT and H...,"Lexington, Massachusetts, United States",Broad Institute of MIT and Harvard,Northeastern University,"Operations Lead in the PRISM group, part of th...",NaN,Broad Institute of MIT and Harvard,NaN,Jan 2019 – Present,3 yrs 1 mo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Operation Lead,Broad Institute of MIT and Harvard,"$62,227"
289,Technical Lead at HubSpot,"Boston, Massachusetts, United States",HubSpot,Northeastern University,NaN,NaN,HubSpot,NaN,Sep 2019 – Present,2 yrs 5 mos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Technical Lead,HubSpot,"$137,928"
290,Senior Strategy Analyst at Tufts Health Plan,"Boston, Massachusetts, United States",Tufts Health Plan,Northeastern University,Experienced Senior Product Analyst with a demo...,NaN,Senior Strategy Analyst,Tufts Health Plan,Jan 2021 – Present,1 yr 1 mo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Senior Strategy Analyst,Tufts Health Plan,"$62,968 - $83,294"
291,Clinical Trial Manager at Apellis,Greater Boston,Apellis Pharmaceuticals,Emmanuel College,Project Manager coming from a background of di...,NaN,Clinical Trial Manager,Apellis Pharmaceuticals,Jun 2019 – Present,2 yrs 8 mos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clinical Trial Manager,Apellis,"$117,309"


In [18]:
df.to_csv('linkedin_with_salaries.csv')